In [2]:
#Training

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

C:\Users\parmi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
PROCESSED_DIR = "../data/processed"
POLLUTANTS = ["co", "pm25", "pm10", "no2", "so2", "o3"]
SEQ_LEN = 14  # number of past days to use for prediction
BATCH_SIZE = 32
EPOCHS = 50
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODELS_DIR = "models"
os.makedirs(MODELS_DIR, exist_ok=True)  # models folder inside POLLUTANT_FORECASTING

In [3]:
#LSTM MODEL
class LSTMPredictor(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # use only last time step
        return out

In [4]:
def prepare_data(series, seq_len):
    X, y = [], []
    for i in range(len(series) - seq_len):
        X.append(series[i:i+seq_len])
        y.append(series[i+seq_len])
    X = np.array(X)
    y = np.array(y)
    return X, y

In [5]:
def train_lstm(model, train_loader, epochs=EPOCHS, lr=LR):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.6f}")

In [6]:
available_files = os.listdir(PROCESSED_DIR)
print(f"Files in processed folder: {available_files}")

for pollutant in POLLUTANTS:
    file_name = f"{pollutant}_clean.csv"
    file_path = os.path.join(PROCESSED_DIR, file_name)

    if file_name not in available_files:
        print(f"File not found: {file_path}. Skipping {pollutant.upper()}.")
        continue

    print(f"\n--- Training LSTM for {pollutant.upper()} ---")
    
    # Load processed data
    df = pd.read_csv(file_path, parse_dates=['date'])
    df = df.sort_values('date')

    # Use normalized value column
    if "value_norm" not in df.columns:
        print(f"'value_norm' column not found in {file_name}. Skipping.")
        continue

    series = df["value_norm"].values.reshape(-1, 1)

    # Prepare sequences
    X, y = prepare_data(series, SEQ_LEN)
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    # Create DataLoader
    dataset = TensorDataset(X, y)
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    # Initialize model
    model = LSTMPredictor(input_dim=1).to(DEVICE)

    # Train model
    train_lstm(model, train_loader)

    # Save model
    torch.save(model.state_dict(), os.path.join(MODELS_DIR, f"lstm_{pollutant}.pt"))
    print(f"{pollutant.upper()} model saved to {MODELS_DIR}/\n")

print("Training complete for all available pollutants.")


Files in processed folder: ['co_clean.csv', 'co_minmax.csv', 'no2_clean.csv', 'no2_minmax.csv', 'o3_clean.csv', 'o3_minmax.csv', 'pm10_clean.csv', 'pm10_minmax.csv', 'pm25_clean.csv', 'pm25_minmax.csv', 'so2_clean.csv', 'so2_minmax.csv']

--- Training LSTM for CO ---
Epoch 10/50, Loss: 0.006842
Epoch 20/50, Loss: 0.002501
Epoch 30/50, Loss: 0.001960
Epoch 40/50, Loss: 0.001928
Epoch 50/50, Loss: 0.001897
CO model saved to models/


--- Training LSTM for PM25 ---
Epoch 10/50, Loss: 0.004956
Epoch 20/50, Loss: 0.002254
Epoch 30/50, Loss: 0.000967
Epoch 40/50, Loss: 0.000512
Epoch 50/50, Loss: 0.000503
PM25 model saved to models/


--- Training LSTM for PM10 ---
Epoch 10/50, Loss: 0.002681
Epoch 20/50, Loss: 0.000831
Epoch 30/50, Loss: 0.000396
Epoch 40/50, Loss: 0.000353
Epoch 50/50, Loss: 0.000347
PM10 model saved to models/


--- Training LSTM for NO2 ---
Epoch 10/50, Loss: 0.006288
Epoch 20/50, Loss: 0.004224
Epoch 30/50, Loss: 0.004179
Epoch 40/50, Loss: 0.004211
Epoch 50/50, Loss: 0

In [1]:
# import os
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, TensorDataset

# # -------------------------------
# # CONFIG
# # -------------------------------
# # Path relative to POLLUTANT_FORECASTING folder
# PROCESSED_DIR = "../data/processed"
# POLLUTANTS = ["co", "pm25", "pm10", "no2", "so2", "o3"]
# SEQ_LEN = 14  # number of past days to use for prediction
# BATCH_SIZE = 32
# EPOCHS = 50
# LR = 0.001
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# MODELS_DIR = "models"
# os.makedirs(MODELS_DIR, exist_ok=True)  # models folder inside POLLUTANT_FORECASTING

# # -------------------------------
# # LSTM MODEL
# # -------------------------------
# class LSTMPredictor(nn.Module):
#     def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=1):
#         super().__init__()
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
    
#     def forward(self, x):
#         out, _ = self.lstm(x)
#         out = self.fc(out[:, -1, :])  # use only last time step
#         return out

# # -------------------------------
# # DATA PREP FUNCTION
# # -------------------------------
# def prepare_data(series, seq_len):
#     X, y = [], []
#     for i in range(len(series) - seq_len):
#         X.append(series[i:i+seq_len])
#         y.append(series[i+seq_len])
#     X = np.array(X)
#     y = np.array(y)
#     return X, y

# # -------------------------------
# # TRAIN FUNCTION
# # -------------------------------
# def train_lstm(model, train_loader, epochs=EPOCHS, lr=LR):
#     criterion = nn.MSELoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     model.train()
#     for epoch in range(epochs):
#         epoch_loss = 0
#         for xb, yb in train_loader:
#             xb, yb = xb.to(DEVICE), yb.to(DEVICE)
#             optimizer.zero_grad()
#             pred = model(xb)
#             loss = criterion(pred, yb)
#             loss.backward()
#             optimizer.step()
#             epoch_loss += loss.item()
#         if (epoch+1) % 10 == 0:
#             print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.6f}")

# # -------------------------------
# # MAIN LOOP: TRAIN FOR EACH POLLUTANT
# # -------------------------------
# # List files in processed folder
# available_files = os.listdir(PROCESSED_DIR)
# print(f"Files in processed folder: {available_files}")

# for pollutant in POLLUTANTS:
#     file_name = f"{pollutant}_clean.csv"
#     file_path = os.path.join(PROCESSED_DIR, file_name)

#     if file_name not in available_files:
#         print(f"File not found: {file_path}. Skipping {pollutant.upper()}.")
#         continue

#     print(f"\n--- Training LSTM for {pollutant.upper()} ---")
    
#     # Load processed data
#     df = pd.read_csv(file_path, parse_dates=['date'])
#     df = df.sort_values('date')

#     # Use normalized value column
#     if "value_norm" not in df.columns:
#         print(f"'value_norm' column not found in {file_name}. Skipping.")
#         continue

#     series = df["value_norm"].values.reshape(-1, 1)

#     # Prepare sequences
#     X, y = prepare_data(series, SEQ_LEN)
#     X = torch.tensor(X, dtype=torch.float32)
#     y = torch.tensor(y, dtype=torch.float32)

#     # Create DataLoader
#     dataset = TensorDataset(X, y)
#     train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

#     # Initialize model
#     model = LSTMPredictor(input_dim=1).to(DEVICE)

#     # Train model
#     train_lstm(model, train_loader)

#     # Save model
#     torch.save(model.state_dict(), os.path.join(MODELS_DIR, f"lstm_{pollutant}.pt"))
#     print(f"{pollutant.upper()} model saved to {MODELS_DIR}/\n")

# print("Training complete for all available pollutants.")


C:\Users\parmi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Files in processed folder: ['co_clean.csv', 'no2_clean.csv', 'o3_clean.csv', 'pm10_clean.csv', 'pm25_clean.csv', 'so2_clean.csv']

--- Training LSTM for CO ---
Epoch 10/50, Loss: 0.007215
Epoch 20/50, Loss: 0.002457
Epoch 30/50, Loss: 0.002109
Epoch 40/50, Loss: 0.002114
Epoch 50/50, Loss: 0.001860
CO model saved to models/


--- Training LSTM for PM25 ---
Epoch 10/50, Loss: 0.003971
Epoch 20/50, Loss: 0.001286
Epoch 30/50, Loss: 0.000593
Epoch 40/50, Loss: 0.000658
Epoch 50/50, Loss: 0.000455
PM25 model saved to models/


--- Training LSTM for PM10 ---
Epoch 10/50, Loss: 0.003805
Epoch 20/50, Loss: 0.001720
Epoch 30/50, Loss: 0.000673
Epoch 40/50, Loss: 0.000419
Epoch 50/50, Loss: 0.000428
PM10 model saved to models/


--- Training LSTM for NO2 ---
Epoch 10/50, Loss: 0.004807
Epoch 20/50, Loss: 0.004122
Epoch 30/50, Loss: 0.004298
Epoch 40/50, Loss: 0.004082
Epoch 50/50, Loss: 0.004142
NO2 model saved to models/


--- Training LSTM for SO2 ---
Epoch 10/50, Loss: 0.009766
Epoch 20/50, 